In [107]:

import warnings
warnings.filterwarnings('ignore')
from numpy import set_printoptions
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
pd.options.display.max_columns=None
import seaborn as sns 
from pandas import read_csv
import io
import base64
import json
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

import os
import os.path

from urllib.error import HTTPError

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier


from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


### CARGUE DE DATOS Y CONEXIÓN A DRIVE

In [108]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/credentials.json', SCOPES) # Reemplazar con la ruta correcta
        creds = flow.run_local_server(port=0)
    with open('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/token.json', 'w') as token:
        token.write(creds.to_json())
        
# Crear una instancia de la API de Drive
drive_service = build('drive', 'v3', credentials=creds)

# ID de la carpeta de Google Drive
folder_id = '1hQeetmO4XIObUefS_nzePqKqq3VksUEC'

# Ruta de destino para guardar los archivos descargados
save_path = 'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS'  # Reemplazar con la ruta deseada

# Función para descargar archivos de la carpeta de Drive
def download_folder(folder_id, save_path):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed=false",
        fields='files(id, name)').execute()
    items = results.get('files', [])
    for item in items:
        file_id = item['id']
        file_name = item['name']
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(os.path.join(save_path, file_name), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Descargando {file_name}: {int(status.progress() * 100)}%")
    print("Descarga completa")

# Descargar archivos de la carpeta
download_folder(folder_id, save_path)

# Listar archivos descargados
files = os.listdir(save_path)
print("Archivos descargados:")
for file in files:
    print(file)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899973764197-biu188dkvsgi2al0fh29udm7keak0lh0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61844%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=l0B0271GA0f478OzZXw1g7vrlEl81d&access_type=offline


Descargando industrial10.csv: 100%
Descargando industrial9.csv: 100%
Descargando industrial8.csv: 100%
Descargando industrial7.csv: 100%
Descargando industrial6.csv: 100%
Descargando industrial5.csv: 100%
Descargando industrial3.csv: 100%
Descargando industrial4.csv: 100%
Descargando industrial2.csv: 100%
Descargando industrial1.csv: 100%
Descarga completa
Archivos descargados:
industrial1.csv
industrial10.csv
industrial2.csv
industrial3.csv
industrial4.csv
industrial5.csv
industrial6.csv
industrial7.csv
industrial8.csv
industrial9.csv


In [ ]:
variables_por_carrera = {
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO'],
        '2': ['variable4_sistemas', 'variable5_sistemas', 'variable6_sistemas']
    },
    'sistemas': {
        '1': ['variable1_industrial', 'variable2_industrial', 'variable3_industrial'],
        '2': ['variable4_industrial', 'variable5_industrial', 'variable6_industrial']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}

In [ ]:
def cargar_datos(carrera, semestre):
    
    ruta_archivo = f'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS/{carrera}{semestre}.csv'
    datos = pd.read_csv(ruta_archivo,sep=";")
    
    return datos
carrera="industrial"
semestre="1"

In [ ]:
datos = cargar_datos(carrera, semestre)

columnas_filtradas = variables_por_carrera[carrera][semestre]
df = datos[columnas_filtradas]
print("DataFrame con columnas filtradas:")
df=df.astype(int)
df

DataFrame con columnas filtradas:


,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,QUIMICA_ICFES,IDIOMA_ICFES,LOCALIDAD,RENDIMIENTO_UNO
0,474,65,73,75,79,20,2
1,523,99,69,78,60,1,2
2,483,63,76,82,77,20,1
3,528,88,69,78,75,19,1
4,478,78,69,66,69,8,2
...,...,...,...,...,...,...,...
2003,342,68,0,0,58,11,2
2004,347,69,0,0,71,19,2
2005,348,67,0,0,68,11,1
2006,340,70,0,0,63,11,2


In [ ]:
X=df.loc[:, ~df.columns.str.contains('RENDIMIENTO')]
Y = df.loc[:, df.columns.str.contains('RENDIMIENTO')]                                                      
print("Separación de datos usando Pandas") 
print(X.shape, Y.shape)

Separación de datos usando Pandas
(2008, 6) (2008, 1)


In [ ]:
X = X.astype('float32')                         
Y = LabelEncoder().fit_transform(Y.astype('str'))                
print(X.shape, Y.shape)

(2008, 6) (2008,)


In [ ]:
from collections import Counter
frecuencias = Counter(Y)
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")

El valor 1 aparece 1249 veces
El valor 0 aparece 675 veces
El valor 2 aparece 83 veces
El valor 3 aparece 1 veces


### TRANSFORMACIÓN DE DATOS

In [ ]:
X_T_JOHNSON1 = X.copy(deep=True)
def transformacion_johnson(X):
    transformador_johnson = PowerTransformer(method='yeo-johnson', standardize=True).fit(X)
    datos_transformados = transformador_johnson.transform(X)
    set_printoptions(precision=3)
    print(datos_transformados[:5, :])
    datos_transformados_df = pd.DataFrame(data=datos_transformados, columns=X.columns)
    return datos_transformados_df
Xpandas_T_JOHNSON1 = transformacion_johnson(X_T_JOHNSON1)
Xpandas_T_JOHNSON1.head(2)

[[ 1.598 -0.897  1.005  1.046  1.268  1.353]
 [ 2.502  2.983  0.97   1.072 -0.304 -2.363]
 [ 1.767 -1.135  1.031  1.104  1.099  1.353]
 [ 2.592  1.759  0.97   1.072  0.931  1.202]
 [ 1.673  0.621  0.97   0.965  0.432 -0.682]]


,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,QUIMICA_ICFES,IDIOMA_ICFES,LOCALIDAD
0,1.597922,-0.896989,1.005440,1.046458,1.268133,1.353436
1,2.501767,2.983213,0.970075,1.071632,-0.304398,-2.362881


In [ ]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
Xpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y))
"""


'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\n# Crear una instancia de RandomOverSampler\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\n# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias\nXpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)\n\n# Imprimir la distribución de clases después del oversampling\nprint("Distribución de clases después del oversampling:", Counter(Y))\n'

In [ ]:
X_trn, X_tst, Y_trn, Y_tst = train_test_split(Xpandas_T_JOHNSON1, Y, test_size=0.3, random_state=2)
print('DATOS: Son {} datos para entrenamiento y {} datos para prueba'.format(X_trn.shape[0], X_tst.shape[0]))

DATOS: Son 1405 datos para entrenamiento y 603 datos para prueba


In [ ]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
X_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y_trn))
"""


'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\n# Crear una instancia de RandomOverSampler\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\n# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias\nX_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)\n\n# Imprimir la distribución de clases después del oversampling\nprint("Distribución de clases después del oversampling:", Counter(Y_trn))\n'

In [ ]:
from collections import Counter
frecuencias = Counter(Y_trn)
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")

El valor 1 aparece 861 veces
El valor 0 aparece 477 veces
El valor 2 aparece 66 veces
El valor 3 aparece 1 veces


### KNEIGHBORSCLASSIFIER

In [ ]:
def entrenar_modelo_knn_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {
        'n_neighbors': [i for i in range(14, 18, 1)],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'algorithm': ['auto'],
        'p': [i for i in range(1, 6)],
        'weights': ['uniform']
    }
    modelo = KNeighborsClassifier()
    semilla = 5
    num_folds = 10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn_transformado, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)

    # Entrenar el modelo con los mejores hiperparámetros
    mejor_modelo = KNeighborsClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_knn = entrenar_modelo_knn_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 59.93161094224925
Mejor valor PARAMETRO usando k-fold: {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 16, 'p': 1, 'weights': 'uniform'}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_knn.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.50792994331564
Exhaustividad (Recall):  0.5787728026533997
Puntuación F1:  0.52877975562655
Exactitud:  0.5787728026533997

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.30      0.16      0.21       198
           1       0.64      0.82      0.72       388
           2       0.00      0.00      0.00        17

    accuracy                           0.58       603
   macro avg       0.31      0.33      0.31       603
weighted avg       0.51      0.58      0.53       603



### SVC

In [ ]:
def entrenar_modelo_svc_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'kernel':  ['rbf'], 
            'C': [i/10000 for i in range(8,12,1)],
            'max_iter':[i for i in range(1,3,1)],
            'gamma' : [i/100 for i in range(90,110,5)],
            'random_state':[i for i in range(1,5,1)]}
    modelo = SVC()
    semilla=5
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = SVC(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_svc = entrenar_modelo_svc_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 59.430597771023294
Mejor valor PARAMETRO usando k-fold: {'C': 0.0008, 'gamma': 1.05, 'kernel': 'rbf', 'max_iter': 1, 'random_state': 1}


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_svc.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.5207791886339446
Exhaustividad (Recall):  0.6069651741293532
Puntuación F1:  0.5336250490854828
Exactitud:  0.6069651741293532

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.33      0.11      0.17       198
           1       0.64      0.89      0.74       388
           2       0.00      0.00      0.00        17

    accuracy                           0.61       603
   macro avg       0.32      0.33      0.30       603
weighted avg       0.52      0.61      0.53       603



### DECISION TREE

In [ ]:
def entrenar_modelo_tree_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {          
            'max_depth':[i for i in range(3,6,1)],
            'min_samples_split' :  [i for i in range(1,3,1)],  
            'min_samples_leaf' : [i for i in range(1,3,1)], 
            'max_features' : [i for i in range(5,7,1)], 
            'splitter': ["best", "random"],
            'random_state': [i for i in range(1,4,1)],
            'criterion': ['entropy']}
    modelo = DecisionTreeClassifier()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = DecisionTreeClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_tree = entrenar_modelo_tree_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.56433637284701
Mejor valor PARAMETRO usando k-fold: {'criterion': 'entropy', 'max_depth': 4, 'max_features': 6, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 1, 'splitter': 'random'}


In [ ]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_tree.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.5300855025473677
Exhaustividad (Recall):  0.6235489220563848
Puntuación F1:  0.5333105484805998
Exactitud:  0.6235489220563848

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.35      0.08      0.13       198
           1       0.65      0.93      0.76       388
           2       0.00      0.00      0.00        17

    accuracy                           0.62       603
   macro avg       0.33      0.34      0.30       603
weighted avg       0.53      0.62      0.53       603



### NAIVE BAYES

In [ ]:
def entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {}
    modelo = GaussianNB()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GaussianNB(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_gaussian = entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 60.85916919959473
Mejor valor PARAMETRO usando k-fold: {}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_gaussian.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.5558536044814331
Exhaustividad (Recall):  0.6401326699834162
Puntuación F1:  0.5191201508406547
Exactitud:  0.6401326699834162

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.43      0.03      0.06       198
           1       0.65      0.98      0.78       388
           2       0.00      0.00      0.00        17

    accuracy                           0.64       603
   macro avg       0.36      0.34      0.28       603
weighted avg       0.56      0.64      0.52       603



### LDA

In [ ]:
def entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'solver':  ['svd','lsqr','eigen'],
            'n_components':[1,2,3,4,5,None],
            'shrinkage': ['auto',None, 0, 0.001, 0.01, 0.1, 0.5,1]}
    modelo = LinearDiscriminantAnalysis()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LinearDiscriminantAnalysis(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_LDA = entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.49848024316109
Mejor valor PARAMETRO usando k-fold: {'n_components': 1, 'shrinkage': 1, 'solver': 'lsqr'}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_LDA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.5244075546333103
Exhaustividad (Recall):  0.6417910447761194
Puntuación F1:  0.5094394653585858
Exactitud:  0.6417910447761194

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.33      0.01      0.02       198
           1       0.64      0.99      0.78       388
           2       0.00      0.00      0.00        17

    accuracy                           0.64       603
   macro avg       0.33      0.33      0.27       603
weighted avg       0.52      0.64      0.51       603



### BAGGINGCLASSIFIER

In [ ]:
from sklearn.ensemble import BaggingClassifier
estimator = BaggingClassifier()
estimator.get_params()

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
def entrenar_modelo_BG_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'n_estimators': [i for i in range(750,760,5)],
            'max_samples' : [i/100.0 for i in range(40,50,5)],
            'max_features': [i/100.0 for i in range(60,65,5)],
            'bootstrap': [True], 
            'bootstrap_features': [True]}
    
    base_estimator= DecisionTreeClassifier(criterion= 'gini', 
                                max_depth=5, max_features= 3,min_samples_leaf= 4, 
                                min_samples_split = 8,random_state= 10, splitter= 'random')

    modelo = BaggingClassifier(estimator=base_estimator, oob_score=True, random_state=1)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = BaggingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo
X_trn = X_trn
Y_trn = Y_trn 
modelo_BG = entrenar_modelo_BG_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.142350557244185
Mejor valor PARAMETRO usando k-fold: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.6, 'max_samples': 0.45, 'n_estimators': 750}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_BG.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.6046107532202811
Exhaustividad (Recall):  0.6517412935323383
Puntuación F1:  0.5627442788824598
Exactitud:  0.6517412935323383

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.55      0.11      0.18       198
           1       0.66      0.96      0.78       388
           2       0.00      0.00      0.00        17

    accuracy                           0.65       603
   macro avg       0.40      0.36      0.32       603
weighted avg       0.60      0.65      0.56       603



### EXTRA TREES CLASSIFIER

In [ ]:
def entrenar_modelo_extra_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'min_samples_split' : [i for i in range(1,3,1)], 
                'min_samples_leaf' : [i for i in range(0,2,1)] }
    
    semilla=7            
    modelo = ExtraTreesClassifier(random_state=semilla, 
                                n_estimators=40, max_features=1,max_depth= 10,
                                min_samples_leaf=1,  min_samples_split = 2,
                                bootstrap=True,criterion='gini') 
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = ExtraTreesClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_extra = entrenar_modelo_extra_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.64032421479231
Mejor valor PARAMETRO usando k-fold: {'min_samples_leaf': 1, 'min_samples_split': 2}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_extra.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.571788606566432
Exhaustividad (Recall):  0.6053067993366501
Puntuación F1:  0.5742745143346262
Exactitud:  0.6053067993366501

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.40      0.26      0.32       198
           1       0.66      0.80      0.73       388
           2       0.50      0.06      0.11        17

    accuracy                           0.61       603
   macro avg       0.52      0.38      0.38       603
weighted avg       0.57      0.61      0.57       603



### ADABOOST

In [ ]:
from sklearn.ensemble import AdaBoostClassifier            
estimator  = AdaBoostClassifier ()
estimator.get_params()

{'algorithm': 'SAMME.R',
 'estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': None}

In [ ]:
def entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'learning_rate' : [i/10000.0 for i in range(5,20,5)]}
    semilla=7            
    modelo = AdaBoostClassifier(estimator = None,  algorithm = 'SAMME.R', 
                                random_state= None, n_estimators =50) 
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = AdaBoostClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_ADA = entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.28368794326241
Mejor valor PARAMETRO usando k-fold: {'learning_rate': 0.001}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_ADA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.41402715554345465
Exhaustividad (Recall):  0.6434494195688225
Puntuación F1:  0.5038514122960709
Exactitud:  0.6434494195688225

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       198
           1       0.64      1.00      0.78       388
           2       0.00      0.00      0.00        17

    accuracy                           0.64       603
   macro avg       0.21      0.33      0.26       603
weighted avg       0.41      0.64      0.50       603



### GRADIENT BOOSTING MACHINE

In [ ]:
def entrenar_modelo_GD_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'subsample' : [ 0.5,0.6,0.7 , 0.75 , 0.8 , 0.85 , 0.9 , 0.95 , 1 ]        
              }
    semilla=7
    modelo = GradientBoostingClassifier(random_state=semilla,
                                    n_estimators= 100,learning_rate= 0.1,max_depth= 2,
                                    min_samples_split= 2, min_samples_leaf= 3,max_features= 2)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GradientBoostingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_GD = entrenar_modelo_GD_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.63931104356638
Mejor valor PARAMETRO usando k-fold: {'subsample': 0.9}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_GD.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.5528933768459033
Exhaustividad (Recall):  0.6185737976782753
Puntuación F1:  0.5600654496079799
Exactitud:  0.6185737976782753

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.40      0.17      0.23       198
           1       0.66      0.88      0.75       388
           2       0.00      0.00      0.00        17

    accuracy                           0.62       603
   macro avg       0.35      0.35      0.33       603
weighted avg       0.55      0.62      0.56       603



### XGBOOST

In [ ]:
def entrenar_modelo_XB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'reg_alpha': [0,0.1,0.2,0.3,0.4,0.5],
                'reg_lambda':  [i/100.0 for i in range(130,150,5)]}
    semilla=7
    modelo = XGBClassifier (random_state=semilla,                       
                        n_estimators=40,colsample_bytree = 1, subsample =1,max_depth =2,
                        min_child_weight =6,gamma = 0.05,learning_rate = 0.3)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = XGBClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_XB = entrenar_modelo_XB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.35258358662614
Mejor valor PARAMETRO usando k-fold: {'reg_alpha': 0.3, 'reg_lambda': 1.4}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_XB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.5229913217157277
Exhaustividad (Recall):  0.5572139303482587
Puntuación F1:  0.5378442260053621
Exactitud:  0.5572139303482587

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.33      0.27      0.30       198
           1       0.65      0.73      0.68       388
           2       0.00      0.00      0.00        17

    accuracy                           0.56       603
   macro avg       0.32      0.33      0.33       603
weighted avg       0.52      0.56      0.54       603



### CATBOOST

In [111]:
def entrenar_modelo_CB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'border_count':[53],'l2_leaf_reg': [42]} 
    semilla=7
    modelo = CatBoostClassifier(random_state=semilla, verbose =0)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = CatBoostClassifier(verbose=0,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_CB = entrenar_modelo_CB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 60.78520770010132
Mejor valor PARAMETRO usando k-fold: {'border_count': 53, 'l2_leaf_reg': 42}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_CB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.5410368802256945
Exhaustividad (Recall):  0.6218905472636815
Puntuación F1:  0.5477270824633331
Exactitud:  0.6218905472636815

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.37      0.12      0.18       198
           1       0.65      0.90      0.76       388
           2       0.00      0.00      0.00        17

    accuracy                           0.62       603
   macro avg       0.34      0.34      0.31       603
weighted avg       0.54      0.62      0.55       603



### LIGHT GBM

In [ ]:
def entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 'min_child_samples' : [i for i in range(200, 10000, 100)]}
    semilla=7
    modelo = LGBMClassifier (random_state=semilla,                           
                        num_leaves =  10,max_depth = 1, n_estimators = 100,    
                        learning_rate = 0.1 ,class_weight=  None, subsample = 1,
                        colsample_bytree= 1, reg_alpha=  0, reg_lambda = 0,
                        min_split_gain = 0, boosting_type = 'gbdt')
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LGBMClassifier(verbose=-1,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_LIGHT = entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 379
[LightGBM] [Info] Number of data points in the train set: 1405, number of used features: 6
[LightGBM] [Info] Start training from score -1.080276
[LightGBM] [Info] Start training from score -0.489698
[LightGBM] [Info] Start training from score -3.058138
[LightGBM] [Info] Start training from score -7.247793
Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.06788247213779
Mejor valor PARAMETRO usando k-fold: {'min_child_samples': 200}


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_LIGHT.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.5505247485201729
Exhaustividad (Recall):  0.6285240464344942
Puntuación F1:  0.5479421916800242
Exactitud:  0.6285240464344942

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.40      0.11      0.17       198
           1       0.65      0.92      0.76       388
           2       0.00      0.00      0.00        17

    accuracy                           0.63       603
   macro avg       0.35      0.34      0.31       603
weighted avg       0.55      0.63      0.55       603



### VOTING - VOTACIÓN DURA

In [ ]:
def entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    
    modelo1 = GradientBoostingClassifier(random_state=semilla, n_estimators=800, 
                                    learning_rate = 0.01, max_depth = 2,
                                    max_features =2, min_samples_leaf = 9, 
                                    min_samples_split = 2, subsample = 1 )
    
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, 
                                    criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')

    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=1200, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,
                                    bootstrap = False, max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=1100, max_features=None,
                                    min_samples_split = 5, max_depth= 3, 
                                    min_samples_leaf= 1)

    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, 
                                    random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9,
                                    max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, random_state= 10, splitter= 'random')

    modelo7 = XGBClassifier(n_estimators= 40, random_state=semilla,colsample_bytree = 1, 
                                    subsample =1, reg_alpha = 0.2, reg_lambda= 1.35,
                                    learning_rate= 0.3,max_depth =2, min_child_weight =6,
                                    gamma = 0.05)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), 
                                    ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6),
                                    ('XGB',modelo7)],voting='hard') 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, 
                                    cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_voting_hard = entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 61.92401215805472  % 


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_voting_hard.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.5598971658449382
Exhaustividad (Recall):  0.6401326699834162
Puntuación F1:  0.5295854720389799
Exactitud:  0.6401326699834162

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.43      0.05      0.09       198
           1       0.65      0.97      0.78       388
           2       0.00      0.00      0.00        17

    accuracy                           0.64       603
   macro avg       0.36      0.34      0.29       603
weighted avg       0.56      0.64      0.53       603



### VOTING - VOTACIÓN SUAVE

In [ ]:
def entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(random_state=semilla, n_estimators=800, 
                                    learning_rate = 0.01, max_depth = 2,
                                    max_features =2, min_samples_leaf = 9, 
                                    min_samples_split = 2, subsample = 1 )
    
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')

    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=1200, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=1100, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)

    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9, max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), ('Extratrees', modelo3),
                                    ('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)],
    voting='soft',weights=[0.9,0.7,0.9,0.9,0.9,0.9]) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_voting_soft = entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 62.279128672745685  % 


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_voting_soft.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.578127879122904
Exhaustividad (Recall):  0.6451077943615257
Puntuación F1:  0.5398875608128101
Exactitud:  0.6451077943615257

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.48      0.07      0.12       198
           1       0.65      0.97      0.78       388
           2       0.00      0.00      0.00        17

    accuracy                           0.65       603
   macro avg       0.38      0.34      0.30       603
weighted avg       0.58      0.65      0.54       603



### BLENDING ( METAMODELO LINEAL)

In [ ]:
def entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')
    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=500, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=300, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)
    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9,
                                    max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, random_state= 10, splitter= 'random')
    estimador_final = LogisticRegression()
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[ ('Adaboost', modelo2), ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_lineal = entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 62.34802431610943  % 


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_stacking_lineal.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.6171040739863294
Exhaustividad (Recall):  0.6567164179104478
Puntuación F1:  0.5642628580817219
Exactitud:  0.6567164179104478

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.58      0.11      0.18       198
           1       0.66      0.97      0.79       388
           2       0.00      0.00      0.00        17

    accuracy                           0.66       603
   macro avg       0.41      0.36      0.32       603
weighted avg       0.62      0.66      0.56       603



### STACKING (METAMODELO NO LINEAL)

In [ ]:
def entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo3 = ExtraTreesClassifier(n_estimators=400, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=500, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)
    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, max_features= None,  min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9, max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                    bootstrap = False, max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[  ('Extratrees', modelo3),('Random Forest',modelo4),('Bagging',modelo5),
                                    ('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_nolineal = entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 61.142350557244185  % 


In [ ]:
# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_stacking_nolineal.predict(X_tst)

precision = precision_score(Y_tst, Y_pred, average='weighted')
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

### SUPER APRENDIZ CON ML ESEMBLE

### SUPER APRENDIZ CON ML ESEMBLE - DOS CAPAS 